In [2]:
import math
import random
import numpy as np
from numba import njit
from numba.typed import List
import time
import copy

In [3]:
# function checks if order must not be changed
@njit(cache=True)
def mustPrecede(cust_i:int,dish_i:int,cust_j:int,dish_j:int):
    if cust_i == cust_j:
        return dish_i < dish_j
    elif cust_i < cust_j:
        return dish_i <= dish_j
    else:
        return dish_j < dish_i and dish_i <= dish_j + (cust_j - cust_i -1)
        
# function to move all customers for a given order to process
@njit(cache=True)
def serveCustomer(n:int, pos_waiter:float, custToServ:int, positions:list[float], orders: list[list[float]], t_now:float, v_waiter:float, v_cust:float, t_serving:float):
    newPositions = [p for p in positions]
    newOrders = List()
    [newOrders.append([o for o in ol]) for ol in orders]
    #newOrders = [[o for o in ol] for ol in orders]

    servedOrderAt = newOrders[custToServ][0]
    delta_t= max(servedOrderAt-newPositions[custToServ]/v_cust, abs(servedOrderAt-pos_waiter)/v_waiter) + t_serving # time required to serve customer
    t_now += delta_t
    pos_waiter = servedOrderAt
    # do not remove order yet as it is used to determine blocking conditions

    # update customer position of each of the n customers
    if len(newOrders[0]) > 0: # check if orders are still in cue
        newPositions[0] = min(newOrders[0][0], newPositions[0] + delta_t * v_cust)
    else:
        newPositions[0] = newPositions[0] + delta_t * v_cust # no further orders

    for i in range(1,n):
        if len(newOrders[i]) > 0: # check if orders are still in cue
            newPositions[i] = min(newOrders[i][0], newPositions[i] + delta_t * v_cust, newPositions[i-1] - 1)
        else:
            newPositions[i] = min(newPositions[i] + delta_t * v_cust, newPositions[i-1] - 1) # no further orders

    servedOrderAt = newOrders[custToServ][0]
    newOrders[custToServ] = newOrders[custToServ][1:]

    return servedOrderAt, pos_waiter, newOrders, newPositions, t_now
    
@njit(cache=True)
def getCandidates(n:int, orders:list[list[float]]):
    candidates=[]
    if len(orders[0]) > 0: # first customer is always possible if order are present
        candidates.append(0)

    for j in range(1,n):
        if len(orders[j]) > 0:
            posCandidate = True
            for i in range(0,j):
                if len(orders[i]) > 0 and mustPrecede(i,orders[i][0],j,orders[j][0]):
                    posCandidate=False
                    break
            if posCandidate:
                candidates.append(j)
    return candidates

In [60]:
class State():
    def __init__(self):
        self.served = []
        self.pos_waiter = 0
        self.positions = []
        self.orders = [[]]
        self.time = 0

# Beamsearch
def beamSearch(beta:int, n:int, pos_waiter:float, positions:list[float], orders:list[list[float]], time:float, v_waiter:float, v_cust:float, t_serving:float) -> list[int]:
    noOfOrders = sum([len(e) for e in orders])

    # make order list numba compatible
    nbList = List()
    [nbList.append(o) for o in orders]

    curLevel = []#  list containing states at current level
    
    # initial state of tree
    s = State()
    s.pos_waiter = pos_waiter
    s.positions = positions
    s.orders = nbList
    s.time = time
    curLevel.append(s)

    for _ in range(noOfOrders):
        newLevel = [] # list containing states at new level
        for state in curLevel:
            candidates=getCandidates(n,state.orders)
            for custToServ in candidates:
                newState = State()
                [servedOrderAt, newState.pos_waiter, newState.orders, newState.positions, newState.time] \
                    = serveCustomer(n, state.pos_waiter, custToServ, state.positions, state.orders, state.time, v_waiter, v_cust, t_serving)
                newState.served = state.served + [[custToServ, servedOrderAt]]
                newLevel.append(newState)
            
        # reduce tree size if necessary
        if len(newLevel) > beta:
            threshold = sorted([state.time for state in newLevel])[beta]
            reducedLevel = list(filter(lambda state: state.time <= threshold, newLevel))
            curLevel = reducedLevel
        else:
            curLevel = newLevel
    if len(curLevel) == 0:
        return None

    result = curLevel[0]
    for s in curLevel[1:]:
        if s.time < result.time:
            result = s
    
    return result

In [83]:
# Grundlage des Codes
# > https://pythonmana.com/2021/05/20210502184221296q.html
# > **https://ofstack.com/python/45529/python-mathematical-modeling-learning-simulated-annealing-algorithm-example-analysis-of-integer-programming-problem.html**

def OptimizationSSA(tInitial, tFinal, alfa, meanMarkov, beta, n, pos_waiter, positions, orders, time, v_waiter, v_cust, t_serving):
    result = beamSearch(beta, n, pos_waiter, positions, orders, time, v_waiter, v_cust, t_serving)

    # ====== Simulated annealing algorithm initialization ======
    ordersNow = orders # current order sequence
    resultNow = result # the current result of the beam search
    bestResult = result # the best result found
    recordIter = [] # Number of external cycles
    recordResultNow = [] # The objective function value of the current solution
    recordBestResult = [] # The objective function value of the best solution
    recordPBad = [] # The acceptance probability of the inferior solution
    kIter = 0 # The number of iterations of the outer loop , The number of temperature states
    totalMar = 0 # A total of Markov Chain length
    totalImprove = 0 # The number of improvements for fxBest
    nMarkov = meanMarkov # Fixed length Markov chain

    # ====== Start simulated annealing optimization ======
    # Outer loop , Until the current temperature reaches the end temperature
    tNow = tInitial # Initialize the current temperature (current temperature)
    while tNow >= tFinal: # Outer loop , Until the current temperature reaches the end temperature
        # At the current temperature , Do it enough times (nMarkov) To achieve thermal equilibrium
        kBetter = 0 # The number of times a good solution is obtained
        kBadAccept = 0 # The number of times a bad solution is accepted
        kBadRefuse = 0 # The number of times a bad solution is rejected
        # --- Inner loop , The number of cycles is Markov Chain length
        for k in range(nMarkov): # Inner loop , The number of cycles is Markov Chain length
            totalMar += 1 # total Markov Chain length counter
            # --- New solutions
            # New solutions ： A new solution is generated by random perturbation near the current solution
            ordersNew = copy.deepcopy(ordersNow)
            v = random.sample(range(n),2) # produce two unique random numbers in range [0,n-1]
            # switch two random orders
            o1 = ordersNow[v[1]]
            o0 = ordersNow[v[0]]
            ordersNew[v[0]] = copy.deepcopy(o1)
            ordersNew[v[1]] = copy.deepcopy(o0)
            # --- Calculate the new time required to finish all orders
            newResult = beamSearch(beta, n, pos_waiter, positions, orders, time, v_waiter, v_cust, t_serving)
            # --- Press Metropolis The criteria accept new interpretations
            # Accept judgment ： according to Metropolis The criteria decide whether to accept the new interpretation
            if newResult.time < resultNow.time: # Better solution ： If the objective function of the new solution is better than the current solution , Then accept the new explanation
                accept = True
                kBetter += 1
            else: # Tolerance solution ： If the objective function of the new solution is worse than the current solution , The new solution is accepted with a certain probability
                deltaT = newResult.time - resultNow.time
                pAccept = math.exp(-deltaT / tNow) # Calculate the state transition probability of the tolerant solution
                if pAccept > random.random():
                    accept = True # Accept the bad solution
                    kBadAccept += 1
                else:
                    accept = False # Refuse inferior solutions
                    kBadRefuse += 1
            # Save the new solution
            if accept == True: # If you accept the new explanation , The new solution is saved as the current solution
                resultNow = newResult
                ordersNow = ordersNew
                if newResult.time < bestResult.time: # If the objective function of the new solution is better than the optimal solution , Then the new solution is saved as the optimal solution
                    bestResult = newResult
                    totalImprove += 1
                    #scale = scale*0.99 # Variable search step size , Gradually reduce the search scope , Improve search accuracy

        # --- Data processing after the end of the inner loop
        # Complete the current temperature search , Save data and output
        pBadAccept = kBadAccept / (kBadAccept + kBadRefuse) # The acceptance probability of the inferior solution
        recordIter.append(kIter) # The current number of external loops
        recordResultNow.append(resultNow) # The objective function value of the current solution
        recordBestResult.append(bestResult) # The objective function value of the best solution
        recordPBad.append(round(pBadAccept, 4)) # The objective function value of the best solution
        # Slow down to a new temperature , The cooling curve ：T(k)=alfa*T(k-1)
        tNow = tNow * alfa
        #kIter = kIter + 1
        #fxBest = cal_Energy(xBest, nVar, kIter) # Because the penalty factor increases after iteration , Then we need to reconstruct the augmented objective function
    # ====== End the simulated annealing process ======
    return totalImprove,kIter,bestResult,recordIter,recordResultNow,recordBestResult,recordPBad



In [84]:

# The main program
def main():
    tInitial = 100.0 # Set the initial annealing temperature (initial temperature)
    tFinal = 1 # Set the ending annealing temperature (stop temperature)
    alfa = 0.98 # Set the cooling parameters ,T(k)=alfa*T(k-1)
    meanMarkov = 100 # Markov Chain length , That is, the number of internal circulation runs

    m = 3 #number of counters
    n = 3 #number of customers

    gamma = [1, 4] # interval for dishes per customer 

    orders=[] #list of all orders
    random.seed(42)
    for _ in range(n):
        orders.append(sorted(random.sample(range(m), random.randint(gamma[0], gamma[1]))))
        #temp=[]
        #for _ in range(random.randint(gamma[0], gamma[1])):
        #    temp.append(random.randint(0,9))
        #orders.append(sorted(temp))
    
    positions = list(range(-1,-(n+1),-1)) # list of customer positions
    pos_waiter = 0 # initial position of waiter
    v_waiter = 1 # waiter speed
    v_cust = 1 # customer speed
    t_serving = 2 # time to serve one customer
    time = 0 # initial time
    beta = 50 # beams search width

    # Simulated annealing algorithm
    [totalImprove,kIter,bestResult,recordIter,recordResultNow,recordBestResult,recordPBad] \
    = OptimizationSSA(tInitial, tFinal, alfa, meanMarkov, beta, n, pos_waiter, positions, orders, time, v_waiter, v_cust, t_serving)


if __name__ == '__main__':
    main()


ValueError: not enough values to unpack (expected 8, got 7)

In [61]:
# Test of Beams Search
n=3
m=4
orders=[[1,3], [1], [2]]
positions = list(range(-1,-(n+1),-1))
pos_waiter = 0
v_waiter = 1
v_cust = 1
t_serving = 2
time = 0
beta = 10
result = beamSearch(beta, n, pos_waiter, positions, orders, time, v_waiter, v_cust, t_serving)
print(f'{result.time}:{result.served}')
print(result.orders)


14.0:[[0, 1], [1, 1], [0, 3], [2, 2]]
[[], [], [], ...]


In [62]:
# Test of serve Customer function
n=3
m=4
orders=[[1,3], [1], [2]]
nbList = List()
[nbList.append(o) for o in orders]
positions = list(range(-1,-4,-1))
pos_waiter = 0
servedOrder=[0,1,0,2]
v_waiter = 1
v_cust = 1
t_serving = 2
time = 0

for custToServ in servedOrder:
    [servedOrderAt, pos_waiter, orders, positions, time] = serveCustomer(n, pos_waiter, custToServ, positions, nbList, time, v_waiter, v_cust, t_serving)
    print(time)

4.0
7.0
9.0
14.0
